In [2]:
from datetime import date, datetime
import altair as alt
import polars as pl
import polars_talib as plta
from securities_load.securities.polar_table_functions import (
    retrieve_ohlcv_using_ticker_ids_and_dates,
)

In [3]:
data = retrieve_ohlcv_using_ticker_ids_and_dates([5219, 5230], "2024-07-01", "2025-06-27")

In [4]:
select_ticker = "AAPL"
smas = (50,200)
emas = (12, 26)
talib5 = data.with_columns(
    pl.col("date").dt.strftime("%Y-%m-%d").alias("date_as_string"),
    *[plta.sma(pl.col("close"),timeperiod=i).over("ticker").alias(f"sma{i:03d}") for i in smas],
    *[plta.ema(pl.col("close"),timeperiod=i).over("ticker").alias(f"ema{i:03d}") for i in emas],
    plta.macd(fastperiod=10, slowperiod=20,signalperiod=5).over("ticker").alias("macd"),
    plta.rsi().over("ticker").alias("rsi"),
    plta.ht_dcperiod().over("ticker").alias("ht_dcp"),
    plta.aroon().over("ticker").alias("aroon"),
    plta.wclprice().over("ticker").alias("wclprice"),
    plta.stoch(pl.col("high"), pl.col("low"), pl.col("close"), fastk_period=14, slowk_period=7, slowd_period=7).over("ticker").alias("stoch"),
    plta.cdl2crows().over("ticker").alias("cdl2crows"),
    plta.cdlhammer().over("ticker").alias("cdlhammer"),
).with_columns(
    pl.col("macd").struct.field("macd"),
    pl.col("macd").struct.field("macdsignal"),
    pl.col("macd").struct.field("macdhist"),
    pl.col("aroon").struct.field("aroondown"),
    pl.col("aroon").struct.field("aroonup"),
    pl.col("stoch").struct.field("slowk"),
    pl.col("stoch").struct.field("slowd"),
).select(
    pl.exclude(["macd", "aroon", "stoch"])
).filter(
    pl.col("ticker") == select_ticker
)

print(talib5.tail())

shape: (5, 24)
┌──────┬────────┬────────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id   ┆ ticker ┆ date       ┆ open    ┆ … ┆ aroondown ┆ aroonup   ┆ slowk     ┆ slowd     │
│ ---  ┆ ---    ┆ ---        ┆ ---     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│ i32  ┆ str    ┆ date       ┆ f64     ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
╞══════╪════════╪════════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 5230 ┆ AAPL   ┆ 2025-06-23 ┆ 201.625 ┆ … ┆ 85.714286 ┆ 14.285714 ┆ 29.415211 ┆ 37.280129 │
│ 5230 ┆ AAPL   ┆ 2025-06-24 ┆ 202.59  ┆ … ┆ 78.571429 ┆ 7.142857  ┆ 29.68777  ┆ 34.206831 │
│ 5230 ┆ AAPL   ┆ 2025-06-25 ┆ 201.45  ┆ … ┆ 71.428571 ┆ 0.0       ┆ 37.153786 ┆ 33.034479 │
│ 5230 ┆ AAPL   ┆ 2025-06-26 ┆ 201.43  ┆ … ┆ 64.285714 ┆ 14.285714 ┆ 41.217771 ┆ 32.681741 │
│ 5230 ┆ AAPL   ┆ 2025-06-27 ┆ 201.89  ┆ … ┆ 57.142857 ┆ 7.142857  ┆ 48.516031 ┆ 34.62457  │
└──────┴────────┴────────────┴─────────┴───┴───────────

In [5]:
talib6 = talib5.with_columns(
    pl.when(pl.col("cdlhammer") == 100.0).then(pl.col("wclprice") * 1.025
        ).when(pl.col("cdlhammer") == -100.0).then(pl.col("wclprice") * 0.975
        ).otherwise(None).alias("hammer")
)

In [63]:
MA_colors = ["dodgerblue", "green", "darkorange", "red", "violet", "blue"]
# alt.renderers.enable('mimetype')
base = alt.Chart(talib6).encode(
    x=alt.X(
        "date:T",
        title=None,
        # timeUnit="yearmonthdate",
        axis=alt.Axis(
            tickColor="#D0D3D3",
            domainColor="#D0D3D3",
            gridColor="#D0D3D3",
            labelFontSize=10,
            titleFontSize=12,
            grid=True,
            labelAngle=-45,
            labels=False,
        ),
    ),
)

# Shadows
rule = base.mark_rule().encode(
    y=alt.Y(
        "low:Q",
        title="Price",
        scale=alt.Scale(zero=False),
        axis=alt.Axis(
            format="$.2f",
            tickColor="#9D9BA1",
            domainColor="#9D9BA1",
            gridColor="#D0D3D3",
            grid=True,
            labelFontSize=10,
            titleFontSize=12,
        ),
    ),
    y2=alt.Y2("high:Q"),
    color=alt.condition(
        "datum.open <= datum.close", alt.value("#23DE0E"), alt.value("#f6102b")
    ),
)

bar = base.mark_bar(size=3
    ).encode(
        y=alt.Y("open:Q"),
        y2=alt.Y2("close:Q"),
        color=alt.condition(
            "datum.open <= datum.close", alt.value("#23DE0E"), alt.value("#f6102b")
        ),
        tooltip=["date:T", "open", "high", "low", "close"],
    ).properties(
        width=1300, height=400, title=f"Candlestick for {select_ticker}")

candlestick = rule + bar

# Simple Moving Average
color_count = 0
for i in smas:
    sma_name = f"sma{i:03d}"
    sma = base.mark_line(color=MA_colors[color_count], strokeWidth=1).encode(
        y=alt.Y(sma_name + ":Q"),
        tooltip=["date", sma_name + ":Q"],
    )
    color_count += 1
    candlestick = candlestick + sma

# Exponential Moving Average
for i in emas:
    ema_name = f"ema{i:03d}"
    ema = base.mark_line(color=MA_colors[color_count], strokeDash=[5, 5], strokeWidth=1).encode(
        y=alt.Y(ema_name + ":Q"),
        tooltip=["date", ema_name + ":Q"],
    )
    color_count += 1
    candlestick = candlestick + ema

hammer = base.mark_point(shape="arrow", angle=180, size=100
    ).encode(
        y=alt.Y("hammer:Q"),
        color=alt.value("#000000")
    )
candlestick = candlestick + hammer

hammer_text = base.mark_text(align="left", baseline="bottom", text="Hammer"
    ).encode(
        y=alt.Y("hammer:Q"),
    )
candlestick = candlestick + hammer_text

volume = alt.Chart(talib5).mark_bar(
    size=3,
    color="dodgerblue"
    ).encode(
        x=alt.X(
            "date:T",
            title="",
            axis=alt.Axis(
                tickColor="#9D9BA1",
                domainColor="#9D9BA1",
                gridColor="#D0D3D3",
                labelFontSize=10,
                titleFontSize=12,
                grid=True,
                labelAngle=-45,
                labels=False),
        ),
        y=alt.Y("volume:Q",
            title="Volume",
            axis=alt.Axis(
                format="f",
                tickColor="#9D9BA1",
                domainColor="#9D9BA1",
                gridColor="#D0D3D3",
                grid=True,
                labelFontSize=10,
                titleFontSize=12,
            ),
        ),
            tooltip=["date:T", "volume:Q"],
    ).properties(width=1300, height=100, title="")


macd = alt.Chart(talib5).encode(
    x=alt.X("date:T",
            title="Date",
            axis=alt.Axis(
                tickColor="#D0D3D3",
                domainColor="#D0D3D3",
                gridColor="#D0D3D3",
                labelFontSize=10,
                titleFontSize=12,
                grid=True,
                labelAngle=-45,
                labels=True,
            ),
        )
).properties(width=1300, height=100, title="")  

macd_line = macd.mark_line(color="dodgerblue", strokeWidth=1).encode(
    y=alt.Y("macd:Q", title="MACD & Signal"),
    tooltip=["date:T", "macd:Q"]
)

signal_line = macd.mark_line(color="darkorange", strokeWidth=1).encode(
    y=alt.Y("macdsignal:Q", title = ""),
    tooltip=["date:T", "macdsignal:Q"]
)

macd_lines = alt.layer(macd_line, signal_line).resolve_scale(x='shared',
    y="shared").properties(
    title="",width=1300, height=100
    )

histogram = macd.mark_bar(size=3).encode(
    y=alt.Y("macdhist:Q", title = "MACD Hist",
        axis=alt.Axis(
            # format="f",
            tickColor="#9D9BA1",
            domainColor="#9D9BA1",
            gridColor="#D0D3D3",
            grid=True,
            labelFontSize=10,
            titleFontSize=12,
        ),
    ),
    color=alt.condition(
        alt.datum.macdhist > 0,  # Positive bars
        alt.value("green"),
        alt.value("red")          # Negative bars
    ),
    tooltip=["date:T", "macdhist:Q"],
)

macd_chart = alt.layer(histogram, macd_lines).resolve_scale(x='shared',
    y="independent").properties(
    title="",width=1300, height=100
    )
    
rsi = alt.Chart(talib5).encode(
    x=alt.X("date:T",
            title="",
            axis=alt.Axis(
                tickColor="#D0D3D3",
                domainColor="#D0D3D3",
                gridColor="#D0D3D3",
                labelFontSize=10,
                titleFontSize=12,
                grid=True,
                labelAngle=-45,
                labels=False,
            ),
        )
).properties(width=1300, height=100, title="")  

rsi_line = rsi.mark_line(color="dodgerblue", strokeWidth=1).encode(
    y=alt.Y("rsi:Q", title="RSI"),
    tooltip=["date:T", "rsi:Q"]
)
yrule_upper = alt.Chart().mark_rule().encode(y=alt.datum(70))

# Horizontal line at y = 5
yrule_upper = alt.Chart().mark_rule(strokeDash=[2, 2], color="#9219E9").encode(y=alt.datum(70))
# yrule_upper = (
#     rsi.mark_rule(color="#1010AA", strokeDash=[12, 6], size=2).encode(y=alt.datum(70))
# )

yrule_lower = alt.Chart().mark_rule(strokeDash=[2, 2], color="#9219E9").encode(y=alt.datum(30))

rsi_fill = rsi.mark_area(color="#9219E9",opacity=0.1).encode(y=alt.datum(30), y2=alt.datum(70))

rsi_chart = rsi_line + yrule_lower + yrule_upper + rsi_fill
# rsi_chart = alt.layer(rsi_line, yrule_upper, yrule_lower).resolve_scale(x='shared',
#     y="shared").properties(
#     title="",width=1300, height=100
#     )

combined_chart = alt.vconcat(candlestick, volume, rsi_chart, macd_chart,).resolve_scale(
    x='shared').configure(background="#d7e9fa") #.interactive()
combined_chart


alt.VConcatChart(...)